# 2022.5#4
为了更容易学习，本周我们从经典的线性神经网络开始完成整个神经网络的训练过程，尽可能理解过程中每一步发生了什么，包括定义简单的神经网络架构、数据处理、指定损失函数和训练模型。

In [ ]:
import torch
from torch import Tensor
import random
from d2l import torch as d2l

In [ ]:
# 生成数据集
#
# 写一个带有噪声的模型来生成一个数据集。目标是通过有限样本的数据集
# 恢复这个模型的参数。
#
# 下面是一个实现：生成一个包含500个样本的数据集，每个样本包括从标准
# 正态分布中取出的两个参数。
def sample_generator(w, b, sample_num):
    # y = Xw + b + \epsilon
    X = torch.normal(0, 1, (sample_num, len(w)))
    y:Tensor = X @ w + b
    # std deviation = -.01
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

model_w = torch.tensor([2.022, 5.22])
model_b = torch.tensor(4.2)
features, labels = sample_generator(model_w, model_b, 500)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.scatter(features[:, (1)].detach().numpy(), labels.detach().numpy(), 1)

In [ ]:
# 读取数据集
# 写一个生成器，每次抽取一小批样本来更新参数
# 具体实现是将下标列表随机打乱，每次步进batch_size
# 取出对应下标的样本
'''
在一些实现中我看到所有矩阵甚至列表运算都先将变量转为
Tensor的情况，虽然感觉现阶段没必要仔细了解，但很好奇
为什么Tensor运算的速度远比for循环快，以及为什么GPU
的并行处理要比CPU快，GPU和CPU的并行处理硬件基础有很大
不同吗？

'''
# 这个生成器是为了理解选择过程实现的，它必须先将所有数据
# 读入内存，计算大量随机数，以及不断进行随机访问。这都会
# 严重影响效率。
def choose_batch(batch_size, features, labels):
    sample_num = len(features)
    # 生成一个目录列表并打乱
    indices = list(range(sample_num))
    random.shuffle(indices)
    for i in range(0, sample_num, batch_size):
        # 注意batch_size不一定整除sample_num
        batch_indeces = indices[i:min(i + batch_size, sample_num)]
        yield features[batch_indeces], labels[batch_indeces]

In [ ]:
# 初始化模型参数
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 定义模型
def linear(X, w, b):
    return X @ w + b

# 定义损失函数：均方损失
def squared_loss(real_y:Tensor, y:Tensor):
    return (real_y - y.reshape(real_y.shape)) ** 2 / 2

# 定义优化算法：随机梯度下降
def grediant_descent(params: list[Tensor], lr: float, batch_size: int):
    with torch.no_grad():
        for param in params:
            # 求导，乘以学习率
            param -= param.grad * lr / batch_size
            param.grad.zero_()

In [ ]:
# 训练过程
# 超参数
lr = 0.03
batch_size = 10
# 训练三个周期
epoch_num = 3
net = linear
loss = squared_loss
opt = grediant_descent

for epoch in range(epoch_num):
    for X, y in choose_batch(batch_size, features, labels):
        l = loss(y, net(X, w, b))
        l.sum().backward()
        opt([w, b], lr, batch_size)     
    # 一个周期结束，计算平均误差
    train_l = loss(labels, net(features, w, b))
    print(f'第{epoch + 1}个周期，损失{float(train_l.mean()):f}')
print(f'w的估计误差: {model_w - w.reshape(model_w.shape)}')
print(f'b的估计误差: {model_b - b}')

自此完成了一个完整神经网络的训练。其中还有一点模糊之处，就是Pytorch的求导到底是如何进行的，backward函数究竟做了什么事。

另外还有在优化算法中不使用torch.no_grad()修饰会报错，暂时没搞懂这个错误为何产生

这个网络勉强能认为是一层，还远远不到”深度“的要求，下周期望搭建起一个多层神经网络